提取JSON数据

In [9]:
import json
import re
from stop_list_0 import closed_class_stop_words as words
from collections import Counter
from OCR_utils import getocr, question_processing, alignScore, editDist, alignScore1


In [2]:
val_data = json.load(open('train.json', 'r',encoding='UTF-8'))
unans = ['unanswerable', 'unsuitable', 'unsuitable image', 'image unsuitable', 'unsure', 'no unsuitable image', 'unsuitable imagine', 'unauunsuit image', 'unknown', 'unknow']
for item in val_data:
    for ans in item['answers']:
        if ans['answer'] in unans:
            ans['answer'] = 'unanswerable'
with open('train.json', 'w') as f:
    json.dump(val_data, f, indent = 4, separators = (',', ':'))
val_data = json.load(open('val.json', 'r',encoding='UTF-8'))
unans = ['unanswerable', 'unsuitable', 'unsuitable image', 'image unsuitable', 'unsure', 'no unsuitable image', 'unsuitable imagine', 'unauunsuit image', 'unknown', 'unknow']
for item in val_data:
    for ans in item['answers']:
        if ans['answer'] in unans:
            ans['answer'] = 'unanswerable'
with open('val.json', 'w') as f:
    json.dump(val_data, f, indent = 4, separators = (',', ':'))

In [3]:
#path1->目标原json文件, path2->目标OCR_RESULT
ocrdic, val_data = getocr('train.json', 'train_ocr_result.json')
with open('train_newocr.json', 'w') as f:
    json.dump(ocrdic, f, indent = 4, separators = (',', ':'))
with open('train_newtag.json', 'w') as f:
    json.dump(val_data, f, indent = 4, separators = (',', ':'))

#path1->目标原json文件, path2->目标OCR_RESULT
ocrdic, val_data = getocr('val.json', 'val_ocr_result.json')
with open('val_newocr.json', 'w') as f:
    json.dump(ocrdic, f, indent = 4, separators = (',', ':'))
with open('val_newtag.json', 'w') as f:
    json.dump(val_data, f, indent = 4, separators = (',', ':'))

# #path1->目标原json文件, path2->目标OCR_RESULT
# ocrdic, val_data = getocr('train_val.json', 'train_val_ocr_result.json')
# with open('train_val_newocr.json', 'w') as f:
#     json.dump(ocrdic, f, indent = 4, separators = (',', ':'))
# with open('train_val_newtag.json', 'w') as f:
#     json.dump(val_data, f, indent = 4, separators = (',', ':'))


提取label

In [4]:
#提取停用词
stoplist = words
# print(stoplist)
keylist = [] #所有问题的关键词列表，未去重
total_Tsent = [] #所有问题的包含关键词的句子列表，去重
val_data = json.load(open('train_newtag.json', 'r',encoding='UTF-8'))
for ques in val_data:
    #去除answer_type = "unanssent = []
    sent = set()
    for i in range(10):
        sent.add(ques['answers'][i]['answer'])
    num = Counter(sent)
    # #选出关键词和包含关键词的句子
    #答案的关键词和包含关键词的句子加入total_Tsent
    total_Tsent.extend(list(sent))
keydict = Counter(total_Tsent) #统计关键词频
# Sort the keydict, and then find its first 3000 words;
keydict = sorted(keydict.items(), key = lambda x:x[1], reverse=True)
# onelis = []
# for it in keydict:
#     if it[1] == 1:
#         onelis.append(it[0])
# count = 0
# key3000 = []
# for it in keydict:
#     count += 1
#     if count <= 3000:
#         key3000.append(it[0])
#     else:
#         break
# imgcount = 0
# for ques in val_data:
#     for ans in ques['answers']:
#         if ans['answer'] in key3000:
#             imgcount += 1
#             break
# print('We select ',len(key3000), ' keys')
# print('Total image number: ', len(val_data))
# print('Lables cover ', imgcount, ' images')
label = []
count = 0
for item in keydict:
    if count == 3000:
        break
    label.append(item[0])
    count += 1
print('Current # of words in label list: %d' %(len(label)))
###
###
### Dump label and its index mapping into JSON files;
###
###
label_dict = {}
for i in range(len(label)):
    label_dict[label[i]] = i
with open('labelmap.json', 'w') as f:
    json.dump(label_dict, f, indent = 4, separators = (',', ':'))
with open('label.json', 'w') as f:
    json.dump(label, f, indent = 4, separators = (',', ':'))

Current # of words in label list: 3000


imagelabel

In [5]:
qid = 0
val_data = json.load(open('train_newtag.json','r',encoding='UTF-8'))
answer = []
print('Total train answers num is: ', len(val_data))
ocrcount= 0
for ques in val_data:
    ansdict = {}
    ansdict["answer_type"] = ques["answer_type"]
    ansdict['img_id'] = ques['image'].replace('.jpg','')
    ansdict['label'] = {}
    ansdict['question_id'] = qid
    qid += 1
    ansdict['sent'] = question_processing(ques['question'])
    sent = []
    for i in range(10):
        sent.append(ques['answers'][i]['answer'])
    num = Counter(sent)
    dictionary = sorted(num.items(), key = lambda x:x[1], reverse=True)
    ###
    ###
    ### Soft annotation here based on word freqiency;
    ###
    ###
    ans_count = 0
    if dictionary[0][0] in ["OCR1", "OCR2", "OCR3"]:
        ocrcount += 1
    for ans in dictionary:
        if ans[0] in label:
            ans_count += ans[1]
    if ans_count == 0:
        continue
    else:
        for ans in dictionary:
            if ans[0] in label:
                ansdict['label'][ans[0]] = float(ans[1] / ans_count)
    answer.append(ansdict)
print('Remain train answers num is: ', len(answer))
print('Train answers max is ocr num is: ', ocrcount)
with open('train_imglabel.json', 'w') as f:
    json.dump(answer, f, indent = 4, separators = (',', ':'))

qid = 0
val_data = json.load(open('val_newtag.json','r',encoding='UTF-8'))
newanswer = []
# answer = []
print('Total val answers num is: ', len(val_data))
vallis = []
ocrcount = 0
for ques in val_data:
    ansdict = {}
    ansdict["answer_type"] = ques["answer_type"]
    ansdict['img_id'] = ques['image'].replace('.jpg','')
    ansdict['label'] = {}
    ansdict['question_id'] = qid
    qid += 1
    ansdict['sent'] = question_processing(ques['question'])
    sent = []
    for i in range(10):
        sent.append(ques['answers'][i]['answer'])
    num = Counter(sent)
    dictionary = sorted(num.items(), key = lambda x:x[1], reverse=True)
    ###
    ###
    ### Soft annotation here based on word freqiency;
    ###
    ###
    ans_count = 0
    if dictionary[0][0] in ["OCR1", "OCR2", "OCR3"]:
        ocrcount += 1
    for ans in dictionary:
        if ans[0] in label:
            ans_count += ans[1]
    if ans_count == 0:
        vallis.append(ques)
    else:
        for ans in dictionary:
            if ans[0] in label:
                ansdict['label'][ans[0]] = float(ans[1] / ans_count)
        answer.append(ansdict)
        newanswer.append(ansdict)
print('Remain val answers num is: ', len(newanswer))
print('Train answers max is ocr num is: ', ocrcount)
with open('val_imglabel.json', 'w') as f:
    json.dump(newanswer, f, indent = 4, separators = (',', ':'))

Total train answers num is:  14991
Remain train answers num is:  14130
Train answers max is ocr num is:  731
Total val answers num is:  2934
Remain val answers num is:  2713
Train answers max is ocr num is:  169


OCR-with-ANS OCR-without-ANS

In [6]:
def getocrwithans(path):
    val_data = json.load(open(path,'r',encoding='UTF-8'))
    for item in val_data:
        if val_data[item] == []:
            continue
        else:
            for data in val_data[item]:
                del data[1]
    return val_data
def getocrwithoutans(path):
    val_data = json.load(open(path,'r',encoding='UTF-8'))
    for item in val_data:
        if val_data[item] == []:
            continue
        else:
            for data in val_data[item]:
                del data[2]
    return val_data

In [7]:
ocr_with_ans = getocrwithans('train_newocr.json')
ocr_without_ans = getocrwithoutans('train_newocr.json')
with open('train_ocr_with_ans.json', 'w') as f:
    json.dump(ocr_with_ans, f, indent = 4, separators = (',', ':'))
with open('train_ocr_without_ans.json', 'w') as f:
    json.dump(ocr_without_ans, f, indent = 4, separators = (',', ':'))
ocr_with_ans = getocrwithans('val_newocr.json')
ocr_without_ans = getocrwithoutans('val_newocr.json')
with open('val_ocr_with_ans.json', 'w') as f:
    json.dump(ocr_with_ans, f, indent = 4, separators = (',', ':'))
with open('val_ocr_without_ans.json', 'w') as f:
    json.dump(ocr_without_ans, f, indent = 4, separators = (',', ':'))

筛选掉OCR

In [39]:
# print(len(label))
for it in label:
    if it in ['OCR1', 'OCR2', 'OCR3']:
        label.remove(it)
label_dict = {}
for i in range(len(label)):
    label_dict[label[i]] = i
with open('labelmap_without_OCR.json', 'w') as f:
    json.dump(label_dict, f, indent = 4, separators = (',', ':'))
with open('label_without_OCR.json', 'w') as f:
    json.dump(label, f, indent = 4, separators = (',', ':'))


In [55]:
val_data = json.load(open('train_imglabel.json','r',encoding='UTF-8'))
ocrlis = ['OCR1', 'OCR2', 'OCR3']
val_data_noocr = []
for item in val_data:    
    middic = {}
    for ans in item['label'].keys():
        if ans not in ocrlis:
            middic[ans] = item['label'][ans]
    if len(middic) != 0:
        item['label'] = middic
        val_data_noocr.append(item)
with open('train_imagelabel_noocr.json', 'w') as f:
    json.dump(val_data_noocr, f, indent = 4, separators = (',', ':'))
val_data = json.load(open('val_imglabel.json','r',encoding='UTF-8'))
ocrlis = ['OCR1', 'OCR2', 'OCR3']
val_data_noocr = []
for item in val_data:    
    middic = {}
    for ans in item['label'].keys():
        if ans not in ocrlis:
            middic[ans] = item['label'][ans]
    if len(middic) != 0:
        item['label'] = middic
        val_data_noocr.append(item)
with open('val_imagelabel_noocr.json', 'w') as f:
    json.dump(val_data_noocr, f, indent = 4, separators = (',', ':'))

In [12]:
resdata = json.load(open('RES/val_predict.json'))
ocrlis = ['OCR1', 'OCR2', 'OCR3']
ocrdata = []
for item in resdata:
    if item['answer'] in ocrlis and item["answer_type"] != "unanswerable":
        ocrdata.append(item)
with open('RES/val_predict.json', 'w') as f:
    json.dump(ocrdata, f, indent = 4, separators = (',', ':'))